# **TP4 - Grupo 4**
Pedro Paulo Costa Pereira - A88062

Tiago André Oliveira Leite - A91693

In [1]:
from z3 import *

# **Problema - Correção de Programas**

Considere o seguinte programa, em Python anotado, para multiplicação de dois inteiros de precisão limitada a 16 bits.  

```python
assume m >= 0 and n >= 0 and r == 0 and x == m and y == n 

0: while y > 0:
1:    if y & 1 == 1: 
2:            y , r  = y-1 , r+x
3:    x , y = x<<1  ,  y>>1
4: assert r == m * n

```

## Variaveis Globais

In [296]:
Bits = 16
Limite = 8190

### Função que declara as variaveis de cada estado

As variáveis são guardadas guardadas num dicionario $s$.
Sendo que representam:
> $s['m']$ ⟶ um bitvector com o valor inicial de x;<br>
> $s['n']$ ⟶ um bitvector com o valor inicial de y;<br>
> $s['x']$ ⟶ um bitvector com o valor de x;<br>
> $s['y']$ ⟶ um bitvector com o valor de y;<br>
> $s['r']$ ⟶ um bitvector com o valor de r;<br>
> $s['pc']$ ⟶ um bitvector com o valor do program counter;<br>

In [317]:
def declare(i):
    s = {}
    s['m'] = BitVec('m'+str(i),Bits)
    s['n'] = BitVec('n'+str(i), Bits)
    s['x'] = BitVec('x'+str(i), Bits)
    s['y'] = BitVec('y'+str(i), Bits)
    s['r'] = BitVec('r'+str(i), Bits)
    s['pc'] = BitVec('pc'+str(i), Bits)
    
    return s

### Função que adiciona as restrições do estado inicial

As restrições no estado inicial são:
> $m \ge 0 \quad \land\ \quad n \ge 0 \quad \land\ \quad r = 0 \quad \land\ \quad x = m \quad \land\ \quad y = n \quad \land\ \quad pc = 0 $

In [318]:
def init(s):
    return And(s['m'] >= 0,s['m']<Limite, s['n'] >= 0,s['n']<Limite, s['r'] == 0, s['x'] == s['m'], s['y'] == s['n'], s['pc'] == 0) 

### Função que adiciona as restrições associadas a cada transição

In [319]:
def trans(s,p):
    t01 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y']>0,s['pc']==0, p['pc']==1, s['r'] == p['r'])
    
    t12 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y'] & 1 == 1,s['pc']==1, p['pc']==2, s['r'] == p['r'])
    
    t23 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], p['y']==s['y']-1,
              s['pc']==2, p['pc']==3, p['r'] == s['r']+s['x'])
    
    t13 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y'] & 1 != 1,s['pc']==1, p['pc']==3, s['r'] == p['r'])
    
    t30 = And(s['m']==p['m'], s['n']==p['n'], p['x']==s['x']<<1, p['y']==s['y']>>1,
              s['pc']==3, p['pc']==0, s['r'] == p['r'])
    
    t04 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y']<=0,s['pc']==0, p['pc']==4, s['r'] == p['r'])
    
    t44 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['pc']==4, p['pc']==4, s['r'] == p['r'])
    
    return Or(t01, t12, t23, t13, t30, t04, t44)

### Função que gera um traço de execução do sistema com k estados

A função vai simular, atravês do solver Z3, uma execução do sistema com **k** estados, $[E_0 .. E_{k-1}]$, e **k-1** transições, com as restrições codificadas anteriormente. Sendo $E_0$ o estado inicial. 

Caso o sistema seja satisfazivel, vão ser imprimidas, estado a estado, os valores das variáveis do sistema.

In [258]:
def gera_traco(declare,init,trans, k):
    s = Solver()
    traco = {}
    for i in range(k):
        traco[i] = declare(i)
    s.add(init(traco[0]))
    for i in range(k-1):
        s.add(trans(traco[i],traco[i+1]))
    status = s.check()
    if status == sat:
        m = s.model()
        for i in range(k):
            for v in traco[i]:
                if traco[i][v].sort() == RealSort():
                    print(v,'=', float(m[traco[i][v]].numerator_as_long())/float(m[traco[i][v]].denominator_as_long()))
                else:
                    print(v,"=",m[traco[i][v]])
            print('')
    elif status == unsat:
        print("Não ha execuções possiveis")
    else:
        print("Resultado impossivel de obter!")

In [268]:
gera_traco(declare,init,trans, 20)

m = 3303
n = 3680
x = 3303
y = 3680
r = 0
pc = 0

m = 3303
n = 3680
x = 3303
y = 3680
r = 0
pc = 1

m = 3303
n = 3680
x = 3303
y = 3680
r = 0
pc = 3

m = 3303
n = 3680
x = 6606
y = 1840
r = 0
pc = 0

m = 3303
n = 3680
x = 6606
y = 1840
r = 0
pc = 1

m = 3303
n = 3680
x = 6606
y = 1840
r = 0
pc = 3

m = 3303
n = 3680
x = 13212
y = 920
r = 0
pc = 0

m = 3303
n = 3680
x = 13212
y = 920
r = 0
pc = 1

m = 3303
n = 3680
x = 13212
y = 920
r = 0
pc = 3

m = 3303
n = 3680
x = 26424
y = 460
r = 0
pc = 0

m = 3303
n = 3680
x = 26424
y = 460
r = 0
pc = 1

m = 3303
n = 3680
x = 26424
y = 460
r = 0
pc = 3

m = 3303
n = 3680
x = 52848
y = 230
r = 0
pc = 0

m = 3303
n = 3680
x = 52848
y = 230
r = 0
pc = 1

m = 3303
n = 3680
x = 52848
y = 230
r = 0
pc = 3

m = 3303
n = 3680
x = 40160
y = 115
r = 0
pc = 0

m = 3303
n = 3680
x = 40160
y = 115
r = 0
pc = 1

m = 3303
n = 3680
x = 40160
y = 115
r = 0
pc = 2

m = 3303
n = 3680
x = 40160
y = 114
r = 40160
pc = 3

m = 3303
n = 3680
x = 14784
y = 57
r = 40160
p

## Provar por indução a terminação deste programa

### Função que prova uma propriedade atraves por indução

In [377]:
def induction_always(declare,init,trans,prop):
    s = Solver()
    state = declare(0)
    s.add(init(state))
    s.add(Not(prop(state)))
    status = s.check()
    assert(status != unknown)
    if (status == sat):
        print("Não é verdade no estado inicial")
        m = s.model()
        for v in state:
            print(v, '=', m[state[v]])
        return
    
    s = Solver()
    pre = declare(0)
    pos = declare(1)
    s.add(init(pre))
    s.add(prop(pre))
    s.add(trans(pre,pos))
    s.add(Not(prop(pos)))
    status = s.check()
    assert(status != unknown)
    if (status == sat):
        print("Não é possivel provar o passo indutivo no estado")
        m = s.model()
        print('pre')
        for v in pre:
            print(v, '=', m[pre[v]])
        print('pos')
        for v in pos:
            print(v, '=', m[pre[v]])
        return
    print("A propriedade é valida!")

### Função que prova uma propriedade por k-indução

### Propriedades do variante

#### Variante

In [378]:
def variante(state):
    return 4*state['y']-state['pc']+4

#### Nâo negativo

In [379]:
def naonegativo(state):
    return variante(state) >=0

In [380]:
induction_always(declare,init,trans, naonegativo)

A propriedade é valida!


#### Descrescente

In [381]:
def decrescente(state):
    proximo = declare(-1)
    decresce = 4*proximo['y'] + 4 -proximo['pc'] < 4*state['y'] + 4 - state['pc']
    zero = 4*proximo['y'] + 4 - proximo['pc'] == 0
    formula = Implies(trans(state,proximo),Or(zero,decresce))
    return (ForAll(list(proximo.values()),formula))

In [382]:
induction_always(declare,init,trans,decrescente)

A propriedade é valida!


#### Util

In [383]:
def utilidade(state):
    return (Implies(state['y'] + 4 - state['pc'] == 0, state['pc'] == 4))

In [384]:
induction_always(declare,init,trans,utilidade)

A propriedade é valida!
